In [2]:
import numpy as np


In [3]:
def normalize(value, range_, ideal):
    min_val, max_val = range_
    if value in ideal:
        return 1.0
    
    if value < min(ideal):
        return 1 - (min(ideal) - value) / (min(ideal) - min_val)
    
    return 1 - (value - max(ideal)) / (max_val - max(ideal))


def normalize_candidates(candidates, ranges, ideal_ref):
    normalized_data = {}
    for candidate, values in candidates.items():
        normalized_values = []
        for i, (criterion, value) in enumerate(zip(ranges.keys(), values)):
            normalized_values.append(normalize(value, ranges[criterion], ideal_ref[criterion]))
        normalized_data[candidate] = normalized_values
    return normalized_data


def weighted_normalized_matrix(normalized_data, weights):
    normalized_array = np.array([values for values in normalized_data.values()])
    weighted_normalized_array = normalized_array * weights
    return weighted_normalized_array


def calculate_I_plus_minus(weighted_normalized_array, weights):
    I_plus = np.sqrt(np.sum((weighted_normalized_array - weights) ** 2, axis=1))
    I_minus = np.sqrt(np.sum(weighted_normalized_array ** 2, axis=1))
    return I_plus, I_minus


def calculate_R(I_plus, I_minus):
    return I_minus / (I_plus + I_minus)


def rank_candidates(candidates, R):
    results = {candidate: r for candidate, r in zip(candidates.keys(), R)}
    sorted_results = sorted(results.items(), key=lambda item: item[1], reverse=True)
    return sorted_results


In [4]:
candidates = {
    'A': [30, 0, 2, 3, 3, 2],
    'B': [40, 9, 1, 3, 2, 2],
    'C': [25, 0, 3, 1, 3, 2],
    'D': [27, 0, 5, 3, 3, 1],
    'E': [45, 15, 2, 2, 3, 4]
}

weights = [0.2262, 0.2143, 0.1786, 0.1429, 0.1190, 0.1190]

ranges = {
    'C1': [23, 60],
    'C2': [0, 15],
    'C3': [0, 10],
    'C4': [1, 3],  # Ruim (1), Razoável (2), Bom (3)
    'C5': [1, 3],  # Total (1), Parcial (2), Sem (3)
    'C6': [1, 5]   # Baixa (1), Normal Baixa (2), Normal (3), Normal Alta (4), Alta (5)
}

ideal_ref = {
    'C1': [30, 35],
    'C2': [10, 15],
    'C3': [0],
    'C4': [3],
    'C5': [3],
    'C6': [4, 5]
}

In [5]:
normalized_data = normalize_candidates(candidates, ranges, ideal_ref)
normalized_data

{'A': [1.0, 0.0, 0.8, 1.0, 1.0, 0.33333333333333337],
 'B': [0.8, 0.9, 0.9, 1.0, 0.5, 0.33333333333333337],
 'C': [0.2857142857142857, 0.0, 0.7, 0.0, 1.0, 0.33333333333333337],
 'D': [0.5714285714285714, 0.0, 0.5, 1.0, 1.0, 0.0],
 'E': [0.6, 1.0, 0.8, 0.5, 1.0, 1.0]}

In [6]:
weighted_normalized_array = weighted_normalized_matrix(normalized_data, weights)
print("Ponderado:\n",weighted_normalized_array)

Ponderado:
 [[0.2262     0.         0.14288    0.1429     0.119      0.03966667]
 [0.18096    0.19287    0.16074    0.1429     0.0595     0.03966667]
 [0.06462857 0.         0.12502    0.         0.119      0.03966667]
 [0.12925714 0.         0.0893     0.1429     0.119      0.        ]
 [0.13572    0.2143     0.14288    0.07145    0.119      0.119     ]]


In [7]:
I_plus, I_minus = calculate_I_plus_minus(weighted_normalized_array, weights)
print("\nI_plus:")
for candidate, value in zip(candidates.keys(), I_plus):
    print(f"{candidate}: {value:.6f}")


I_plus:
A: 0.231288
B: 0.112512
C: 0.318771
D: 0.278313
E: 0.120697


In [8]:
print("\nI_minus:")
for candidate, value in zip(candidates.keys(), I_minus):
    print(f"{candidate}: {value:.6f}")


I_minus:
A: 0.328232
B: 0.348306
C: 0.188524
D: 0.243441
E: 0.343782


In [9]:
R = calculate_R(I_plus, I_minus)
print("\nR:")
for candidate, value in zip(candidates.keys(), R):
    print(f"{candidate}: {value:.6f}")


R:
A: 0.586631
B: 0.755843
C: 0.371626
D: 0.466582
E: 0.740146


In [10]:
sorted_results = rank_candidates(candidates, R)
for candidate, score in sorted_results:
    print(f'Candidato {candidate}: {score:.4f}')

Candidato B: 0.7558
Candidato E: 0.7401
Candidato A: 0.5866
Candidato D: 0.4666
Candidato C: 0.3716
